# Language Translation
In this project, you’re going to take a peek into the realm of neural network machine translation.  You’ll be training a sequence to sequence model on a dataset of English and French sentences that can translate new sentences from English to French.
## Get the Data
Since translating the whole language of English to French will take lots of time to train, we have provided you with a small portion of the English corpus.

In [3]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import problem_unittests as tests

source_path = 'data/small_vocab_en'
target_path = 'data/small_vocab_fr'
source_text = helper.load_data(source_path)
target_text = helper.load_data(target_path)

## Explore the Data
Play around with view_sentence_range to view different parts of the data.

In [4]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in source_text.split()})))

sentences = source_text.split('\n')
word_counts = [len(sentence.split()) for sentence in sentences]
print('Number of sentences: {}'.format(len(sentences)))
print('Average number of words in a sentence: {}'.format(np.average(word_counts)))

print()
print('English sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(source_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))
print()
print('French sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(target_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 227
Number of sentences: 137861
Average number of words in a sentence: 13.225277634719028

English sentences 0 to 10:
new jersey is sometimes quiet during autumn , and it is snowy in april .
the united states is usually chilly during july , and it is usually freezing in november .
california is usually quiet during march , and it is usually hot in june .
the united states is sometimes mild during june , and it is cold in september .
your least liked fruit is the grape , but my least liked is the apple .
his favorite fruit is the orange , but my favorite is the grape .
paris is relaxing during december , but it is usually chilly in july .
new jersey is busy during spring , and it is never hot in march .
our least liked fruit is the lemon , but my least liked is the grape .
the united states is sometimes busy during january , and it is sometimes warm in november .

French sentences 0 to 10:
new jersey est parfois calme pendant l' automne 

## Implement Preprocessing Function
### Text to Word Ids
As you did with other RNNs, you must turn the text into a number so the computer can understand it. In the function `text_to_ids()`, you'll turn `source_text` and `target_text` from words to ids.  However, you need to add the `<EOS>` word id at the end of `target_text`.  This will help the neural network predict when the sentence should end.

You can get the `<EOS>` word id by doing:
```python
target_vocab_to_int['<EOS>']
```
You can get other word ids using `source_vocab_to_int` and `target_vocab_to_int`.

In [5]:
def text_to_ids(source_text, target_text, source_vocab_to_int, target_vocab_to_int):
    """
    Convert source and target text to proper word ids
    :param source_text: String that contains all the source text.
    :param target_text: String that contains all the target text.
    :param source_vocab_to_int: Dictionary to go from the source words to an id
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: A tuple of lists (source_id_text, target_id_text)
    """
    # Split first the source and targets in sentences
    source_sentences = source_text.split('\n')
    target_sentences = target_text.split('\n')
    
    # Create the lists of numbers
    source_id_text =[]
    target_id_text = []
    
    # List for sources
    for source_sentence in source_sentences:
        source_id_text.append([source_vocab_to_int[word] for word in source_sentence.split()])
    
    # List for targets
    for index, target_sentence in enumerate(target_sentences):
        target_id_text.append([target_vocab_to_int[word] for word in target_sentence.split()])
        # Add the end of sentence at the end of the target id
        target_id_text[index].extend([target_vocab_to_int['<EOS>']])

    return source_id_text, target_id_text

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_text_to_ids(text_to_ids)

Tests Passed


### Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
helper.preprocess_and_save_data(source_path, target_path, text_to_ids)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [7]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np
import helper
import problem_unittests as tests

(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()

### Check the Version of TensorFlow and Access to GPU
This will check to make sure you have the correct version of TensorFlow and access to a GPU

In [8]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf
from tensorflow.python.layers.core import Dense

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.1'), 'Please use TensorFlow version 1.1 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.3.0
Default GPU Device: /gpu:0


## Build the Neural Network
You'll build the components necessary to build a Sequence-to-Sequence model by implementing the following functions below:
- `model_inputs`
- `process_decoder_input`
- `encoding_layer`
- `decoding_layer_train`
- `decoding_layer_infer`
- `decoding_layer`
- `seq2seq_model`

### Input
Implement the `model_inputs()` function to create TF Placeholders for the Neural Network. It should create the following placeholders:

- Input text placeholder named "input" using the TF Placeholder name parameter with rank 2.
- Targets placeholder with rank 2.
- Learning rate placeholder with rank 0.
- Keep probability placeholder named "keep_prob" using the TF Placeholder name parameter with rank 0.
- Target sequence length placeholder named "target_sequence_length" with rank 1
- Max target sequence length tensor named "max_target_len" getting its value from applying tf.reduce_max on the target_sequence_length placeholder. Rank 0.
- Source sequence length placeholder named "source_sequence_length" with rank 1

Return the placeholders in the following the tuple (input, targets, learning rate, keep probability, target sequence length, max target sequence length, source sequence length)

In [9]:
def model_inputs():
    """
    Create TF Placeholders for input, targets, learning rate, and lengths of source and target sequences.
    :return: Tuple (input, targets, learning rate, keep probability, target sequence length,
    max target sequence length, source sequence length)
    """
    # Define placeholders for each of the parameters
    inputs = tf.placeholder(tf.int32, (None, None), name='input')
    targets = tf.placeholder(tf.int32, (None, None), name='targets')
    learning_rate = tf.placeholder(tf.float32, name='learning_rate')
    keep_probability = tf.placeholder(tf.float32, name='keep_prob')
    target_sequence_length = tf.placeholder(tf.int32, (None,), name='target_sequence_length')
    max_target_len = tf.reduce_max(target_sequence_length)
    source_sequence_length = tf.placeholder(tf.int32, (None,), name='source_sequence_length')
    
    
    return inputs, targets, learning_rate, keep_probability, target_sequence_length, max_target_len, source_sequence_length


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_model_inputs(model_inputs)

ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.framework.ops.Operation'>):
<tf.Operation 'assert_rank_2/Assert/Assert' type=Assert>
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
['File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main\n    "__main__", mod_spec)', 'File "/usr/lib/python3.5/runpy.py", line 85, in _run_code\n    exec(code, run_globals)', 'File "/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py", line 16, in <module>\n    app.launch_new_instance()', 'File "/usr/local/lib/python3.5/dist-packages/traitlets/config/application.py", line 658, in launch_instance\n    app.start()', 'File "/usr/local/lib/python3.5/dist-packages/ipykernel/kernelapp.py", line 477, in start\n    ioloop.IOLoop.instance().start()', 'File "/usr/local/lib/python3.5/dist-packages/zmq/eventloop/ioloop.py", line 177, in start\n    super(ZMQIOLoop, self).start()', 'File "/usr/lo

ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.framework.ops.Operation'>):
<tf.Operation 'assert_rank_2/Assert/Assert' type=Assert>
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
['File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main\n    "__main__", mod_spec)', 'File "/usr/lib/python3.5/runpy.py", line 85, in _run_code\n    exec(code, run_globals)', 'File "/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py", line 16, in <module>\n    app.launch_new_instance()', 'File "/usr/local/lib/python3.5/dist-packages/traitlets/config/application.py", line 658, in launch_instance\n    app.start()', 'File "/usr/local/lib/python3.5/dist-packages/ipykernel/kernelapp.py", line 477, in start\n    ioloop.IOLoop.instance().start()', 'File "/usr/local/lib/python3.5/dist-packages/zmq/eventloop/ioloop.py", line 177, in start\n    super(ZMQIOLoop, self).start()', 'File "/usr/lo

ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.framework.ops.Operation'>):
<tf.Operation 'assert_rank_3/Assert/Assert' type=Assert>
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
['File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main\n    "__main__", mod_spec)', 'File "/usr/lib/python3.5/runpy.py", line 85, in _run_code\n    exec(code, run_globals)', 'File "/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py", line 16, in <module>\n    app.launch_new_instance()', 'File "/usr/local/lib/python3.5/dist-packages/traitlets/config/application.py", line 658, in launch_instance\n    app.start()', 'File "/usr/local/lib/python3.5/dist-packages/ipykernel/kernelapp.py", line 477, in start\n    ioloop.IOLoop.instance().start()', 'File "/usr/local/lib/python3.5/dist-packages/zmq/eventloop/ioloop.py", line 177, in start\n    super(ZMQIOLoop, self).start()', 'File "/usr/lo

ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.framework.ops.Operation'>):
<tf.Operation 'assert_rank_3/Assert/Assert' type=Assert>
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
['File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main\n    "__main__", mod_spec)', 'File "/usr/lib/python3.5/runpy.py", line 85, in _run_code\n    exec(code, run_globals)', 'File "/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py", line 16, in <module>\n    app.launch_new_instance()', 'File "/usr/local/lib/python3.5/dist-packages/traitlets/config/application.py", line 658, in launch_instance\n    app.start()', 'File "/usr/local/lib/python3.5/dist-packages/ipykernel/kernelapp.py", line 477, in start\n    ioloop.IOLoop.instance().start()', 'File "/usr/local/lib/python3.5/dist-packages/zmq/eventloop/ioloop.py", line 177, in start\n    super(ZMQIOLoop, self).start()', 'File "/usr/lo

Tests Passed


### Process Decoder Input
Implement `process_decoder_input` by removing the last word id from each batch in `target_data` and concat the GO ID to the begining of each batch.

In [10]:
def process_decoder_input(target_data, target_vocab_to_int, batch_size):
    """
    Preprocess target data for encoding
    :param target_data: Target Placehoder
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param batch_size: Batch Size
    :return: Preprocessed target data
    """
    # Process the decoder input for its use
    rem_end = tf.strided_slice(target_data, [0,0], [batch_size, -1], [1,1])
    add_go = tf.concat([tf.fill([batch_size, 1], target_vocab_to_int['<GO>']),rem_end], axis=1)
    
    return add_go

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_process_encoding_input(process_decoder_input)

Tests Passed


### Encoding
Implement `encoding_layer()` to create a Encoder RNN layer:
 * Embed the encoder input using [`tf.contrib.layers.embed_sequence`](https://www.tensorflow.org/api_docs/python/tf/contrib/layers/embed_sequence)
 * Construct a [stacked](https://github.com/tensorflow/tensorflow/blob/6947f65a374ebf29e74bb71e36fd82760056d82c/tensorflow/docs_src/tutorials/recurrent.md#stacking-multiple-lstms) [`tf.contrib.rnn.LSTMCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/LSTMCell) wrapped in a [`tf.contrib.rnn.DropoutWrapper`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/DropoutWrapper)
 * Pass cell and embedded input to [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)

In [11]:
from imp import reload
reload(tests)

def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob, 
                   source_sequence_length, source_vocab_size, 
                   encoding_embedding_size):
    """
    Create encoding layer
    :param rnn_inputs: Inputs for the RNN
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param keep_prob: Dropout keep probability
    :param source_sequence_length: a list of the lengths of each sequence in the batch
    :param source_vocab_size: vocabulary size of source data
    :param encoding_embedding_size: embedding size of source data
    :return: tuple (RNN output, RNN state)
    """
    # First lets embed the input
    embed_input = tf.contrib.layers.embed_sequence(rnn_inputs, source_vocab_size, encoding_embedding_size)
    
    # Once the input has been embedded, create the RNN cell of the encoder (tf 1.1, with a def)
    def define_cell(rnn_size):
        cell = tf.contrib.rnn.LSTMCell(rnn_size, initializer=tf.random_uniform_initializer(-0.1, 0.1))
        cell = tf.contrib.rnn.DropoutWrapper(cell, output_keep_prob=keep_prob)
        return cell
    
    # Create the stacked RNN and pass it
    stacked_cell = tf.contrib.rnn.MultiRNNCell([define_cell(rnn_size) for c in range(num_layers)])
    encoded_output, encoded_state = tf.nn.dynamic_rnn(stacked_cell, embed_input, sequence_length=source_sequence_length,
                                                     dtype=tf.float32)
    
    return encoded_output, encoded_state

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_encoding_layer(encoding_layer)

Tests Passed


### Decoding - Training
Create a training decoding layer:
* Create a [`tf.contrib.seq2seq.TrainingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/TrainingHelper) 
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [12]:

def decoding_layer_train(encoder_state, dec_cell, dec_embed_input, 
                         target_sequence_length, max_summary_length, 
                         output_layer, keep_prob):
    """
    Create a decoding layer for training
    :param encoder_state: Encoder State
    :param dec_cell: Decoder RNN Cell
    :param dec_embed_input: Decoder embedded input
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_summary_length: The length of the longest sequence in the batch
    :param output_layer: Function to apply the output layer
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing training logits and sample_id
    """
    # Training helper
    training_helper = tf.contrib.seq2seq.TrainingHelper(dec_embed_input,target_sequence_length, time_major=False)
    # Next the Basic Decoder
    basic_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell, training_helper, encoder_state, output_layer)
    # Finally obtain the decoder outputs, impute finished as we wait for full sentences, keep just the final outputs
    decoder_outputs = tf.contrib.seq2seq.dynamic_decode(basic_decoder, impute_finished=True, 
                                                        maximum_iterations=max_summary_length)[0]
    return decoder_outputs



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_train(decoding_layer_train)

Tests Passed


### Decoding - Inference
Create inference decoder:
* Create a [`tf.contrib.seq2seq.GreedyEmbeddingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/GreedyEmbeddingHelper)
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [13]:
def decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id,
                         end_of_sequence_id, max_target_sequence_length,
                         vocab_size, output_layer, batch_size, keep_prob):
    """
    Create a decoding layer for inference
    :param encoder_state: Encoder state
    :param dec_cell: Decoder RNN Cell
    :param dec_embeddings: Decoder embeddings
    :param start_of_sequence_id: GO ID
    :param end_of_sequence_id: EOS Id
    :param max_target_sequence_length: Maximum length of target sequences
    :param vocab_size: Size of decoder/target vocabulary
    :param decoding_scope: TenorFlow Variable Scope for decoding
    :param output_layer: Function to apply the output layer
    :param batch_size: Batch size
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing inference logits and sample_id
    """
    # First create the inference helper
    # For the start tokens create a tensor of the appropriate dim
    start_tokens = tf.tile(tf.constant([start_of_sequence_id], dtype=tf.int32), [batch_size], name='start_tokens')
    inference_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(dec_embeddings, start_tokens, end_of_sequence_id)
    # The Basic decoder
    basic_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell, inference_helper, encoder_state, output_layer)
    # Again, the decoder outputs, similar to the train layer
    decoder_outputs = tf.contrib.seq2seq.dynamic_decode(basic_decoder, impute_finished=True, 
                                                        maximum_iterations=max_target_sequence_length)[0]
    
    return decoder_outputs



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_infer(decoding_layer_infer)

Tests Passed


### Build the Decoding Layer
Implement `decoding_layer()` to create a Decoder RNN layer.

* Embed the target sequences
* Construct the decoder LSTM cell (just like you constructed the encoder cell above)
* Create an output layer to map the outputs of the decoder to the elements of our vocabulary
* Use the your `decoding_layer_train(encoder_state, dec_cell, dec_embed_input, target_sequence_length, max_target_sequence_length, output_layer, keep_prob)` function to get the training logits.
* Use your `decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id, end_of_sequence_id, max_target_sequence_length, vocab_size, output_layer, batch_size, keep_prob)` function to get the inference logits.

Note: You'll need to use [tf.variable_scope](https://www.tensorflow.org/api_docs/python/tf/variable_scope) to share variables between training and inference.

In [14]:
def decoding_layer(dec_input, encoder_state,
                   target_sequence_length, max_target_sequence_length,
                   rnn_size,
                   num_layers, target_vocab_to_int, target_vocab_size,
                   batch_size, keep_prob, decoding_embedding_size):
    """
    Create decoding layer
    :param dec_input: Decoder input
    :param encoder_state: Encoder state
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_target_sequence_length: Maximum length of target sequences
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param target_vocab_size: Size of target vocabulary
    :param batch_size: The size of the batch
    :param keep_prob: Dropout keep probability
    :param decoding_embedding_size: Decoding embedding size
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    # Embed the target sequences, Create a variable tensor, initialize and use it
    dec_embeddings = tf.Variable(tf.random_uniform([target_vocab_size, decoding_embedding_size]))
    dec_embeds = tf.nn.embedding_lookup(dec_embeddings, dec_input)
    
    # Decoder cell, as above
    def define_cell(rnn_size):
        cell = tf.contrib.rnn.LSTMCell(rnn_size, initializer=tf.random_uniform_initializer(-0.1, 0.1))
        cell = tf.contrib.rnn.DropoutWrapper(cell, output_keep_prob=keep_prob)
        return cell
    
    # Create the stacked RNN and obtain the output, in this case a Dense layer is applied
    stacked_cell = tf.contrib.rnn.MultiRNNCell([define_cell(rnn_size) for c in range(num_layers)])
    output_layer = Dense(target_vocab_size, kernel_initializer= tf.truncated_normal_initializer(mean = 0.0, stddev=0.1))
    
    # In order to share variables, we use a scope
    with tf.variable_scope("decoder"):
        training_decoder_output = decoding_layer_train(encoder_state, stacked_cell, dec_embeds, target_sequence_length, 
                                                       max_target_sequence_length, output_layer, keep_prob)
    
    # Reuse the variables here
    with tf.variable_scope("decoder", reuse=True):
        inference_decoder_output = decoding_layer_infer(encoder_state, stacked_cell, dec_embeddings, 
                                                        target_vocab_to_int['<GO>'], target_vocab_to_int['<EOS>'],
                                                        max_target_sequence_length, target_vocab_size,
                                                        output_layer, batch_size, keep_prob)
    
    return training_decoder_output, inference_decoder_output



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer(decoding_layer)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:

- Encode the input using your `encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob,  source_sequence_length, source_vocab_size, encoding_embedding_size)`.
- Process target data using your `process_decoder_input(target_data, target_vocab_to_int, batch_size)` function.
- Decode the encoded input using your `decoding_layer(dec_input, enc_state, target_sequence_length, max_target_sentence_length, rnn_size, num_layers, target_vocab_to_int, target_vocab_size, batch_size, keep_prob, dec_embedding_size)` function.

In [15]:
def seq2seq_model(input_data, target_data, keep_prob, batch_size,
                  source_sequence_length, target_sequence_length,
                  max_target_sentence_length,
                  source_vocab_size, target_vocab_size,
                  enc_embedding_size, dec_embedding_size,
                  rnn_size, num_layers, target_vocab_to_int):
    """
    Build the Sequence-to-Sequence part of the neural network
    :param input_data: Input placeholder
    :param target_data: Target placeholder
    :param keep_prob: Dropout keep probability placeholder
    :param batch_size: Batch Size
    :param source_sequence_length: Sequence Lengths of source sequences in the batch
    :param target_sequence_length: Sequence Lengths of target sequences in the batch
    :param source_vocab_size: Source vocabulary size
    :param target_vocab_size: Target vocabulary size
    :param enc_embedding_size: Decoder embedding size
    :param dec_embedding_size: Encoder embedding size
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    # Ignoring the encoder output, as explained in seq2seq as is a similar case
    _, enc_state = encoding_layer(input_data, rnn_size, num_layers, keep_prob, source_sequence_length,
                              source_vocab_size, enc_embedding_size)
    
    # First feed the decoder in training mode
    dec_input = process_decoder_input(target_data, target_vocab_to_int, batch_size)
    
    # Now we can pass the encoder and the encoded inputs
    training_decoder_output, inference_decoder_output = decoding_layer(dec_input, enc_state, target_sequence_length,
                                                                       max_target_sentence_length, rnn_size, num_layers, 
                                                                       target_vocab_to_int, target_vocab_size, batch_size,
                                                                       keep_prob, dec_embedding_size) 
    
    return training_decoder_output, inference_decoder_output


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_seq2seq_model(seq2seq_model)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `num_layers` to the number of layers.
- Set `encoding_embedding_size` to the size of the embedding for the encoder.
- Set `decoding_embedding_size` to the size of the embedding for the decoder.
- Set `learning_rate` to the learning rate.
- Set `keep_probability` to the Dropout keep probability
- Set `display_step` to state how many steps between each debug output statement

In [17]:
# Number of Epochs
epochs = 60
# Batch Size
batch_size = 256
# RNN Size
rnn_size = 30
# Number of Layers
num_layers = 2
# Embedding Size
encoding_embedding_size = 20
decoding_embedding_size = 20
# Learning Rate
learning_rate = 0.001
# Dropout Keep Probability
keep_probability = 0.5
display_step = 20

### Build the Graph
Build the graph using the neural network you implemented.

In [18]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
save_path = 'checkpoints/dev'
(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()
max_target_sentence_length = max([len(sentence) for sentence in source_int_text])

train_graph = tf.Graph()
with train_graph.as_default():
    input_data, targets, lr, keep_prob, target_sequence_length, max_target_sequence_length, source_sequence_length = model_inputs()

    #sequence_length = tf.placeholder_with_default(max_target_sentence_length, None, name='sequence_length')
    input_shape = tf.shape(input_data)

    train_logits, inference_logits = seq2seq_model(tf.reverse(input_data, [-1]),
                                                   targets,
                                                   keep_prob,
                                                   batch_size,
                                                   source_sequence_length,
                                                   target_sequence_length,
                                                   max_target_sequence_length,
                                                   len(source_vocab_to_int),
                                                   len(target_vocab_to_int),
                                                   encoding_embedding_size,
                                                   decoding_embedding_size,
                                                   rnn_size,
                                                   num_layers,
                                                   target_vocab_to_int)


    training_logits = tf.identity(train_logits.rnn_output, name='logits')
    inference_logits = tf.identity(inference_logits.sample_id, name='predictions')

    masks = tf.sequence_mask(target_sequence_length, max_target_sequence_length, dtype=tf.float32, name='masks')

    with tf.name_scope("optimization"):
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(
            training_logits,
            targets,
            masks)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(lr)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)


Batch and pad the source and target sequences

In [19]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def pad_sentence_batch(sentence_batch, pad_int):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [pad_int] * (max_sentence - len(sentence)) for sentence in sentence_batch]


def get_batches(sources, targets, batch_size, source_pad_int, target_pad_int):
    """Batch targets, sources, and the lengths of their sentences together"""
    for batch_i in range(0, len(sources)//batch_size):
        start_i = batch_i * batch_size

        # Slice the right amount for the batch
        sources_batch = sources[start_i:start_i + batch_size]
        targets_batch = targets[start_i:start_i + batch_size]

        # Pad
        pad_sources_batch = np.array(pad_sentence_batch(sources_batch, source_pad_int))
        pad_targets_batch = np.array(pad_sentence_batch(targets_batch, target_pad_int))

        # Need the lengths for the _lengths parameters
        pad_targets_lengths = []
        for target in pad_targets_batch:
            pad_targets_lengths.append(len(target))

        pad_source_lengths = []
        for source in pad_sources_batch:
            pad_source_lengths.append(len(source))

        yield pad_sources_batch, pad_targets_batch, pad_source_lengths, pad_targets_lengths


### Train
Train the neural network on the preprocessed data. If you have a hard time getting a good loss, check the forms to see if anyone is having the same problem.

In [ ]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def get_accuracy(target, logits):
    """
    Calculate accuracy
    """
    max_seq = max(target.shape[1], logits.shape[1])
    if max_seq - target.shape[1]:
        target = np.pad(
            target,
            [(0,0),(0,max_seq - target.shape[1])],
            'constant')
    if max_seq - logits.shape[1]:
        logits = np.pad(
            logits,
            [(0,0),(0,max_seq - logits.shape[1])],
            'constant')

    return np.mean(np.equal(target, logits))

# Split data to training and validation sets
train_source = source_int_text[batch_size:]
train_target = target_int_text[batch_size:]
valid_source = source_int_text[:batch_size]
valid_target = target_int_text[:batch_size]
(valid_sources_batch, valid_targets_batch, valid_sources_lengths, valid_targets_lengths ) = next(get_batches(valid_source,
                                                                                                             valid_target,
                                                                                                             batch_size,
                                                                                                             source_vocab_to_int['<PAD>'],
                                                                                                             target_vocab_to_int['<PAD>']))                                                                                                  
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(epochs):
        for batch_i, (source_batch, target_batch, sources_lengths, targets_lengths) in enumerate(
                get_batches(train_source, train_target, batch_size,
                            source_vocab_to_int['<PAD>'],
                            target_vocab_to_int['<PAD>'])):

            _, loss = sess.run(
                [train_op, cost],
                {input_data: source_batch,
                 targets: target_batch,
                 lr: learning_rate,
                 target_sequence_length: targets_lengths,
                 source_sequence_length: sources_lengths,
                 keep_prob: keep_probability})


            if batch_i % display_step == 0 and batch_i > 0:


                batch_train_logits = sess.run(
                    inference_logits,
                    {input_data: source_batch,
                     source_sequence_length: sources_lengths,
                     target_sequence_length: targets_lengths,
                     keep_prob: 1.0})


                batch_valid_logits = sess.run(
                    inference_logits,
                    {input_data: valid_sources_batch,
                     source_sequence_length: valid_sources_lengths,
                     target_sequence_length: valid_targets_lengths,
                     keep_prob: 1.0})

                train_acc = get_accuracy(target_batch, batch_train_logits)

                valid_acc = get_accuracy(valid_targets_batch, batch_valid_logits)

                print('Epoch {:>3} Batch {:>4}/{} - Train Accuracy: {:>6.4f}, Validation Accuracy: {:>6.4f}, Loss: {:>6.4f}'
                      .format(epoch_i, batch_i, len(source_int_text) // batch_size, train_acc, valid_acc, loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_path)
    print('Model Trained and Saved')

Epoch   0 Batch   20/538 - Train Accuracy: 0.2623, Validation Accuracy: 0.3157, Loss: 5.2821
Epoch   0 Batch   40/538 - Train Accuracy: 0.3049, Validation Accuracy: 0.3157, Loss: 4.2463
Epoch   0 Batch   60/538 - Train Accuracy: 0.2373, Validation Accuracy: 0.3159, Loss: 3.9682
Epoch   0 Batch   80/538 - Train Accuracy: 0.2682, Validation Accuracy: 0.3450, Loss: 3.8005
Epoch   0 Batch  100/538 - Train Accuracy: 0.2771, Validation Accuracy: 0.3455, Loss: 3.7104
Epoch   0 Batch  120/538 - Train Accuracy: 0.2689, Validation Accuracy: 0.3455, Loss: 3.6458
Epoch   0 Batch  140/538 - Train Accuracy: 0.2807, Validation Accuracy: 0.3461, Loss: 3.5869
Epoch   0 Batch  160/538 - Train Accuracy: 0.3006, Validation Accuracy: 0.3461, Loss: 3.4255
Epoch   0 Batch  180/538 - Train Accuracy: 0.3142, Validation Accuracy: 0.3461, Loss: 3.3374
Epoch   0 Batch  200/538 - Train Accuracy: 0.2791, Validation Accuracy: 0.3468, Loss: 3.4187
Epoch   0 Batch  220/538 - Train Accuracy: 0.3092, Validation Accuracy

Epoch   3 Batch  240/538 - Train Accuracy: 0.3090, Validation Accuracy: 0.3070, Loss: 1.5001
Epoch   3 Batch  260/538 - Train Accuracy: 0.3008, Validation Accuracy: 0.3141, Loss: 1.4629
Epoch   3 Batch  280/538 - Train Accuracy: 0.2939, Validation Accuracy: 0.3180, Loss: 1.4057
Epoch   3 Batch  300/538 - Train Accuracy: 0.3246, Validation Accuracy: 0.3466, Loss: 1.3994
Epoch   3 Batch  320/538 - Train Accuracy: 0.3560, Validation Accuracy: 0.3917, Loss: 1.4172
Epoch   3 Batch  340/538 - Train Accuracy: 0.3223, Validation Accuracy: 0.3954, Loss: 1.4961
Epoch   3 Batch  360/538 - Train Accuracy: 0.3404, Validation Accuracy: 0.4141, Loss: 1.4733
Epoch   3 Batch  380/538 - Train Accuracy: 0.3963, Validation Accuracy: 0.4300, Loss: 1.4216
Epoch   3 Batch  400/538 - Train Accuracy: 0.3666, Validation Accuracy: 0.4258, Loss: 1.4031
Epoch   3 Batch  420/538 - Train Accuracy: 0.3611, Validation Accuracy: 0.4339, Loss: 1.4317
Epoch   3 Batch  440/538 - Train Accuracy: 0.3932, Validation Accuracy

Epoch   6 Batch  460/538 - Train Accuracy: 0.5236, Validation Accuracy: 0.5362, Loss: 1.0837
Epoch   6 Batch  480/538 - Train Accuracy: 0.5151, Validation Accuracy: 0.5332, Loss: 1.0836
Epoch   6 Batch  500/538 - Train Accuracy: 0.5289, Validation Accuracy: 0.5227, Loss: 1.0365
Epoch   6 Batch  520/538 - Train Accuracy: 0.5166, Validation Accuracy: 0.5376, Loss: 1.1310
Epoch   7 Batch   20/538 - Train Accuracy: 0.5251, Validation Accuracy: 0.5440, Loss: 1.0883
Epoch   7 Batch   40/538 - Train Accuracy: 0.5515, Validation Accuracy: 0.5341, Loss: 1.0009
Epoch   7 Batch   60/538 - Train Accuracy: 0.5223, Validation Accuracy: 0.5419, Loss: 1.1017
Epoch   7 Batch   80/538 - Train Accuracy: 0.4844, Validation Accuracy: 0.5417, Loss: 1.1314
Epoch   7 Batch  100/538 - Train Accuracy: 0.5225, Validation Accuracy: 0.5433, Loss: 1.0663
Epoch   7 Batch  120/538 - Train Accuracy: 0.5041, Validation Accuracy: 0.5371, Loss: 1.0846
Epoch   7 Batch  140/538 - Train Accuracy: 0.4939, Validation Accuracy

Epoch  10 Batch  160/538 - Train Accuracy: 0.5874, Validation Accuracy: 0.5637, Loss: 0.9272
Epoch  10 Batch  180/538 - Train Accuracy: 0.6222, Validation Accuracy: 0.5698, Loss: 0.9274
Epoch  10 Batch  200/538 - Train Accuracy: 0.5576, Validation Accuracy: 0.5676, Loss: 0.9598
Epoch  10 Batch  220/538 - Train Accuracy: 0.5372, Validation Accuracy: 0.5664, Loss: 0.9533
Epoch  10 Batch  240/538 - Train Accuracy: 0.5498, Validation Accuracy: 0.5716, Loss: 0.9713
Epoch  10 Batch  260/538 - Train Accuracy: 0.5601, Validation Accuracy: 0.5662, Loss: 0.9546
Epoch  10 Batch  280/538 - Train Accuracy: 0.5984, Validation Accuracy: 0.5641, Loss: 0.9055
Epoch  10 Batch  300/538 - Train Accuracy: 0.5930, Validation Accuracy: 0.5705, Loss: 0.9041
Epoch  10 Batch  320/538 - Train Accuracy: 0.5688, Validation Accuracy: 0.5710, Loss: 0.9335
Epoch  10 Batch  340/538 - Train Accuracy: 0.5568, Validation Accuracy: 0.5707, Loss: 0.9944
Epoch  10 Batch  360/538 - Train Accuracy: 0.5590, Validation Accuracy

Epoch  13 Batch  380/538 - Train Accuracy: 0.5938, Validation Accuracy: 0.5922, Loss: 0.8746
Epoch  13 Batch  400/538 - Train Accuracy: 0.5835, Validation Accuracy: 0.5856, Loss: 0.8872
Epoch  13 Batch  420/538 - Train Accuracy: 0.5756, Validation Accuracy: 0.6007, Loss: 0.8765
Epoch  13 Batch  440/538 - Train Accuracy: 0.5977, Validation Accuracy: 0.6005, Loss: 0.9421
Epoch  13 Batch  460/538 - Train Accuracy: 0.6075, Validation Accuracy: 0.6037, Loss: 0.8613
Epoch  13 Batch  480/538 - Train Accuracy: 0.6094, Validation Accuracy: 0.5978, Loss: 0.8455
Epoch  13 Batch  500/538 - Train Accuracy: 0.6055, Validation Accuracy: 0.5945, Loss: 0.8226
Epoch  13 Batch  520/538 - Train Accuracy: 0.5752, Validation Accuracy: 0.5888, Loss: 0.8978
Epoch  14 Batch   20/538 - Train Accuracy: 0.6016, Validation Accuracy: 0.6044, Loss: 0.8874
Epoch  14 Batch   40/538 - Train Accuracy: 0.6161, Validation Accuracy: 0.6026, Loss: 0.8095
Epoch  14 Batch   60/538 - Train Accuracy: 0.5918, Validation Accuracy

...

Epoch  37 Batch  360/538 - Train Accuracy: 0.6889, Validation Accuracy: 0.6911, Loss: 0.7144
Epoch  37 Batch  380/538 - Train Accuracy: 0.7051, Validation Accuracy: 0.7044, Loss: 0.6997
Epoch  37 Batch  400/538 - Train Accuracy: 0.6802, Validation Accuracy: 0.6921, Loss: 0.7020
Epoch  37 Batch  420/538 - Train Accuracy: 0.7020, Validation Accuracy: 0.6964, Loss: 0.7066
Epoch  37 Batch  440/538 - Train Accuracy: 0.6979, Validation Accuracy: 0.6951, Loss: 0.7362
Epoch  37 Batch  460/538 - Train Accuracy: 0.6737, Validation Accuracy: 0.6918, Loss: 0.6932
Epoch  37 Batch  480/538 - Train Accuracy: 0.6962, Validation Accuracy: 0.6985, Loss: 0.7019
Epoch  37 Batch  500/538 - Train Accuracy: 0.7058, Validation Accuracy: 0.7122, Loss: 0.6627
Epoch  37 Batch  520/538 - Train Accuracy: 0.6744, Validation Accuracy: 0.6962, Loss: 0.7341
Epoch  38 Batch   20/538 - Train Accuracy: 0.6895, Validation Accuracy: 0.6942, Loss: 0.7030
Epoch  38 Batch   40/538 - Train Accuracy: 0.6946, Validation Accuracy

Epoch  41 Batch   60/538 - Train Accuracy: 0.7262, Validation Accuracy: 0.7054, Loss: 0.6882
Epoch  41 Batch   80/538 - Train Accuracy: 0.6668, Validation Accuracy: 0.6916, Loss: 0.7146
Epoch  41 Batch  100/538 - Train Accuracy: 0.7297, Validation Accuracy: 0.7008, Loss: 0.6689
Epoch  41 Batch  120/538 - Train Accuracy: 0.6947, Validation Accuracy: 0.7022, Loss: 0.6864
Epoch  41 Batch  140/538 - Train Accuracy: 0.6721, Validation Accuracy: 0.6948, Loss: 0.7162
Epoch  41 Batch  160/538 - Train Accuracy: 0.6907, Validation Accuracy: 0.6944, Loss: 0.6517
Epoch  41 Batch  180/538 - Train Accuracy: 0.7199, Validation Accuracy: 0.7029, Loss: 0.6735
Epoch  41 Batch  200/538 - Train Accuracy: 0.6979, Validation Accuracy: 0.6983, Loss: 0.6769
Epoch  41 Batch  220/538 - Train Accuracy: 0.6910, Validation Accuracy: 0.6935, Loss: 0.6694
Epoch  41 Batch  240/538 - Train Accuracy: 0.7041, Validation Accuracy: 0.7026, Loss: 0.6976
Epoch  41 Batch  260/538 - Train Accuracy: 0.6728, Validation Accuracy

Epoch  44 Batch  280/538 - Train Accuracy: 0.7331, Validation Accuracy: 0.7083, Loss: 0.6332
Epoch  44 Batch  300/538 - Train Accuracy: 0.7290, Validation Accuracy: 0.7205, Loss: 0.6622
Epoch  44 Batch  320/538 - Train Accuracy: 0.7042, Validation Accuracy: 0.6974, Loss: 0.6621
Epoch  44 Batch  340/538 - Train Accuracy: 0.7131, Validation Accuracy: 0.7125, Loss: 0.6819
Epoch  44 Batch  360/538 - Train Accuracy: 0.7096, Validation Accuracy: 0.7077, Loss: 0.6756
Epoch  44 Batch  380/538 - Train Accuracy: 0.7227, Validation Accuracy: 0.7209, Loss: 0.6597
Epoch  44 Batch  400/538 - Train Accuracy: 0.7022, Validation Accuracy: 0.7076, Loss: 0.6739
Epoch  44 Batch  420/538 - Train Accuracy: 0.7320, Validation Accuracy: 0.7143, Loss: 0.6645
Epoch  44 Batch  440/538 - Train Accuracy: 0.7115, Validation Accuracy: 0.7180, Loss: 0.7104
Epoch  44 Batch  460/538 - Train Accuracy: 0.6949, Validation Accuracy: 0.7154, Loss: 0.6612
Epoch  44 Batch  480/538 - Train Accuracy: 0.7078, Validation Accuracy

Epoch  47 Batch  500/538 - Train Accuracy: 0.7386, Validation Accuracy: 0.7168, Loss: 0.5964
Epoch  47 Batch  520/538 - Train Accuracy: 0.6947, Validation Accuracy: 0.7198, Loss: 0.6839
Epoch  48 Batch   20/538 - Train Accuracy: 0.7253, Validation Accuracy: 0.7069, Loss: 0.6419
Epoch  48 Batch   40/538 - Train Accuracy: 0.7365, Validation Accuracy: 0.7156, Loss: 0.5835
Epoch  48 Batch   60/538 - Train Accuracy: 0.7480, Validation Accuracy: 0.7280, Loss: 0.6426
Epoch  48 Batch   80/538 - Train Accuracy: 0.6953, Validation Accuracy: 0.7195, Loss: 0.6767
Epoch  48 Batch  100/538 - Train Accuracy: 0.7420, Validation Accuracy: 0.7193, Loss: 0.6341
Epoch  48 Batch  120/538 - Train Accuracy: 0.7121, Validation Accuracy: 0.7138, Loss: 0.6352
Epoch  48 Batch  140/538 - Train Accuracy: 0.7043, Validation Accuracy: 0.7177, Loss: 0.6874
Epoch  48 Batch  160/538 - Train Accuracy: 0.7167, Validation Accuracy: 0.7203, Loss: 0.6169
Epoch  48 Batch  180/538 - Train Accuracy: 0.7405, Validation Accuracy

Epoch  51 Batch  200/538 - Train Accuracy: 0.7189, Validation Accuracy: 0.7250, Loss: 0.6265
Epoch  51 Batch  220/538 - Train Accuracy: 0.7137, Validation Accuracy: 0.7232, Loss: 0.6071
Epoch  51 Batch  240/538 - Train Accuracy: 0.7021, Validation Accuracy: 0.7182, Loss: 0.6452
Epoch  51 Batch  260/538 - Train Accuracy: 0.6979, Validation Accuracy: 0.7338, Loss: 0.6401
Epoch  51 Batch  280/538 - Train Accuracy: 0.7316, Validation Accuracy: 0.7216, Loss: 0.6072
Epoch  51 Batch  300/538 - Train Accuracy: 0.7426, Validation Accuracy: 0.7250, Loss: 0.6117
Epoch  51 Batch  320/538 - Train Accuracy: 0.7238, Validation Accuracy: 0.7266, Loss: 0.6233
Epoch  51 Batch  340/538 - Train Accuracy: 0.7279, Validation Accuracy: 0.7255, Loss: 0.6539
Epoch  51 Batch  360/538 - Train Accuracy: 0.7160, Validation Accuracy: 0.7248, Loss: 0.6491
Epoch  51 Batch  380/538 - Train Accuracy: 0.7271, Validation Accuracy: 0.7225, Loss: 0.6290
Epoch  51 Batch  400/538 - Train Accuracy: 0.7212, Validation Accuracy

### Save Parameters
Save the `batch_size` and `save_path` parameters for inference.

In [ ]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params(save_path)

# Checkpoint

In [ ]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, (source_vocab_to_int, target_vocab_to_int), (source_int_to_vocab, target_int_to_vocab) = helper.load_preprocess()
load_path = helper.load_params()

## Sentence to Sequence
To feed a sentence into the model for translation, you first need to preprocess it.  Implement the function `sentence_to_seq()` to preprocess new sentences.

- Convert the sentence to lowercase
- Convert words into ids using `vocab_to_int`
 - Convert words not in the vocabulary, to the `<UNK>` word id.

In [ ]:
def sentence_to_seq(sentence, vocab_to_int):
    """
    Convert a sentence to a sequence of ids
    :param sentence: String
    :param vocab_to_int: Dictionary to go from the words to an id
    :return: List of word ids
    """
    # Convert to lower case and make a list
    sentence = sentence.lower().split()
    
    # Then transform words into ids
    sentence_to_id = [vocab_to_int.get(word, vocab_to_int['<UNK>']) for word in sentence]
    
    return sentence_to_id


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_sentence_to_seq(sentence_to_seq)

## Translate
This will translate `translate_sentence` from English to French.

In [ ]:
translate_sentence = 'he saw a old yellow truck .'


"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
translate_sentence = sentence_to_seq(translate_sentence, source_vocab_to_int)

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_path + '.meta')
    loader.restore(sess, load_path)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    target_sequence_length = loaded_graph.get_tensor_by_name('target_sequence_length:0')
    source_sequence_length = loaded_graph.get_tensor_by_name('source_sequence_length:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')

    translate_logits = sess.run(logits, {input_data: [translate_sentence]*batch_size,
                                         target_sequence_length: [len(translate_sentence)*2]*batch_size,
                                         source_sequence_length: [len(translate_sentence)]*batch_size,
                                         keep_prob: 1.0})[0]

print('Input')
print('  Word Ids:      {}'.format([i for i in translate_sentence]))
print('  English Words: {}'.format([source_int_to_vocab[i] for i in translate_sentence]))

print('\nPrediction')
print('  Word Ids:      {}'.format([i for i in translate_logits]))
print('  French Words: {}'.format(" ".join([target_int_to_vocab[i] for i in translate_logits])))


## Imperfect Translation
You might notice that some sentences translate better than others.  Since the dataset you're using only has a vocabulary of 227 English words of the thousands that you use, you're only going to see good results using these words.  For this project, you don't need a perfect translation. However, if you want to create a better translation model, you'll need better data.

You can train on the [WMT10 French-English corpus](http://www.statmt.org/wmt10/training-giga-fren.tar).  This dataset has more vocabulary and richer in topics discussed.  However, this will take you days to train, so make sure you've a GPU and the neural network is performing well on dataset we provided.  Just make sure you play with the WMT10 corpus after you've submitted this project.
## Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_language_translation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.